# 此腳本為測試 stence-tranformers 文章向量的效果

In [2]:
from pathlib import Path
import json
import pickle

import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [3]:
root_path = Path('/Users/zoe/Desktop/Kevin_project/judicial_analysis')
input_path = root_path / '108/upload10812'
supreme_court_path = root_path / '199601/最高法院民事'

#### step1. load civial code judical include 車禍 (supreme + 14,904 normal)

In [31]:
accident_judicial = []

for file in supreme_court_path.glob('*'):
    with open(file, 'r') as f:
        judicial_json = json.loads(f.read())
    
    if '車禍' in judicial_json['JFULL']:
        accident_judicial.append(judicial_json)

#### step2. check cosine similarity of vectors with whole judgement & split judgement with dot (。)

In [32]:
with open('all_judicial.pickle', 'rb') as f:
    normal_judicial_json = pickle.load(f)

accident_judicial.extend([ {
    'JFULL': x['JFULLX']['JFULLCONTENT']} 
    for x in normal_judicial_json 
    if x.get('JFULLX') ])
    #if x.get('JFULLX') and not ('車禍' in x['JFULLX']['JFULLCONTENT'])])

In [33]:
len(normal_judicial_json)

14904

#### 1. 經100筆判決書測試後發現，將有提到車禍的判決，與query判決進行比對，會因為判決很長，稀釋了語義，若用句號切斷，相似度會較為明顯，如下案例，
#### 整體文本相似度為 0.1258，但部分段落，有相似度高達 0.6以上，所以未來應該會儲存每段向量、整體向量，並做一些EDA，確認好的評判方式。

### 案例
##### Split corpus similarity:  [tensor([[0.6438]]), '汽\r\n    車行駛時，駕駛人應注意車前狀況及兩車並行之間隔，並隨\r\n    時採取必要之安全措施；道路交通安全規則第94條第1項、\r\n    第3項前段分別定有明文']
##### Split corpus similarity:  [tensor([[0.6339]]), '又被告過失\r\n    行為與系爭機車損害間具有相當因果關係，自應負侵權行為\r\n    損害賠償責任']
##### Split corpus similarity:  [tensor([[0.6187]]), '又汽車在同一車道行\r\n    駛時，除擬超越前車外，後車與前車之間應保持隨時可以煞\r\n    停之距離，不得任意以迫近或其他方式，迫使前車讓道']
##### 判決書有「車禍」字串
##### Whole corpus similarity: 
##### tensor([[0.1258]])

#### 2. 以前100筆判決書觀察，高相似度(cosine similarity)的結果，會提到 車 與 負面意涵(事故/痛苦等)，符合 車禍，車 + 禍害 的文字意涵，但也有不準的時候，
#### 例如: Split corpus similarity:  [tensor([[0.6030]]), '\r\n三、原告所主張之事實，業據其提出與所述相符之系爭契約、系\r\n    爭車輛新領牌照登記書及行車執照、支票及退票理由單、存\r\n    證信函及回執、票據明細查詢表（見本院卷第7 頁至第9 頁\r\n    、第74頁至第77頁、第91頁至第97頁）']

#### 所以下一步應該是如上所述，EDA後，進行100筆標記，評估各種方案的成效(CTR, NDCG等指標)。

In [35]:
model = SentenceTransformer('all-MiniLM-L6-v2')
query_embedding = model.encode('車禍', convert_to_tensor=True)
all_vecs = []

for idx, case in enumerate(accident_judicial):
    show_flag = False
    if idx % 100 == 0:
        print(idx)

    text_sim_sort = sorted([[util.cos_sim(query_embedding, model.encode(paragraph, convert_to_tensor=True)), paragraph]
    for paragraph in case['JFULL'].split('。')], reverse=True)

    for x in text_sim_sort:
        if x[0] > 0.6:
            print('Split corpus similarity: ', x)
            show_flag = True
    if show_flag:
        if '車禍' in case['JFULL']:
            print('判決書有「車禍」字串')
        else:
            print('判決書無「車禍」字串')
        print('Whole corpus similarity: ')
        print(util.cos_sim(query_embedding, model.encode(case['JFULL'], convert_to_tensor=True)))
        print('*'*100)

    all_vecs.extend(text_sim_sort)

0
Split corpus similarity:  [tensor([[0.6178]]), '然\r\n    參酌原告自陳：系爭汽車係被告蔡秀美透過車行購買等語（\r\n    見本院卷第75頁）；又被告間就系爭汽車係成立有償的買賣\r\n    關係等情，亦如前述']
Split corpus similarity:  [tensor([[0.6134]]), '並聲明：被告就系爭汽車之所有權移轉行為（原告就\r\n    被告間系爭汽車之買賣行為，未聲明撤銷）應予撤銷']
Split corpus similarity:  [tensor([[0.6075]]), '\r\n  ㈣綜上所述，原告依據民法第244條第2項規定，請求撤銷被告\r\n    間就系爭汽車之所有權移轉行為，為無理由，應予駁回']
判決書無「車禍」字串
Whole corpus similarity: 
tensor([[0.3542]])
****************************************************************************************************
Split corpus similarity:  [tensor([[0.6049]]), '三、基於轄\r\n    區來往查核之機動性及便利性，報請搭乘自有汽車執行查核\r\n    作業，所需之交通費用請准以汽油收據核銷']
判決書無「車禍」字串
Whole corpus similarity: 
tensor([[0.1259]])
****************************************************************************************************
Split corpus similarity:  [tensor([[0.6438]]), '汽\r\n    車行駛時，駕駛人應注意車前狀況及兩車並行之間隔，並隨\r\n    時採取必要之安全措施；道路交通安全規則第94條第1項、\r\n    第3項前段分別定有明文']
Split corpus similarity:  [tensor([[0.6339]]), '又被告過失\r\n    行為與系爭

In [36]:
all_vecs_df = pd.DataFrame([ float(x[0][0][0]) for x in all_vecs], columns=['vec'])
all_vecs_df.describe()

,vec
count,23307.000000
mean,0.301264
std,0.085484
min,-0.015458
25%,0.250720
50%,0.299657
75%,0.344262
max,0.699597


In [37]:
all_vecs_df.to_excel('vecs_v2.xlsx')

### 相似度分佈，呈現常態分佈，含有車禍的判決書，超過0.4相似度的比例較高。

![](vec_bins_all.png)

# 未實作部分

![test22](dev_spec2.png)